the missing value is on AFRM , it's for the next quarter it's not labeled +1q that's the reason behind that.

In [ ]:
import time as _tm
import requests as _rq
import pandas as _pd
import openpyxl
API_TOKEN = "67ffece4b2ae08.94077168"
BASE_URL  = "https://eodhd.com/api"
HEADERS   = {"User-Agent": "EPS-Trend/1.1"}
PAUSE_SEC = 1.1                     


def _get_json(url: str) -> dict:
    r = _rq.get(url, headers=HEADERS, timeout=30)
    r.raise_for_status()
    return r.json()


def _pull_trend(ticker: str) -> dict:

    url = (
        f"{BASE_URL}/fundamentals/{ticker}"
        f"?filter=Earnings::Trend"           
        f"&api_token={API_TOKEN}&fmt=json"
    )
    data = _get_json(url)

    if "Earnings" in data:          
        return data.get("Earnings", {}).get("Trend", {})
    return data                     


def _latest_snapshots(trend: dict) -> dict[str, dict]:
    latest = {}
    for date_str, rec in trend.items():
        p = rec.get("period")
        if not p:
            continue
        ts = _pd.to_datetime(date_str, errors="coerce")
        if p not in latest or ts > latest[p]["_ts"]:
            rec["_ts"] = ts
            latest[p] = rec
    return latest


def _row_for(ticker: str) -> dict:
    trend = _pull_trend(ticker)
    if not trend:
        return {"Ticker": ticker.split('.', 1)[0], "ERROR": "no Trend"}

    snap = _latest_snapshots(trend)

    periods = {
        "Curr Qtr": "0q",
        "Next Qtr": "+1q",
        "Curr Yr":  "0y",
        "Next Yr":  "+1y",
    }

    def val(rec, fld):
        return float(rec.get(fld)) if rec and rec.get(fld) not in (None, "") else None

    row = {"Ticker": ticker.split('.', 1)[0]}
    for label, code in periods.items():
        rec = snap.get(code, {})
        row[f"{label} Curr"] = val(rec, "epsTrendCurrent")
        row[f"{label} 7"]    = val(rec, "epsTrend7daysAgo")
        row[f"{label} 30"]   = val(rec, "epsTrend30daysAgo")
        row[f"{label} 60"]   = val(rec, "epsTrend60daysAgo")
        row[f"{label} 90"]   = val(rec, "epsTrend90daysAgo")
    return row


def build_table(tickers: list[str]) -> _pd.DataFrame:
    rows = []
    for tk in tickers:
        print(f"{tk} ...", end="", flush=True)
        try:
            rows.append(_row_for(tk))
            print("ok")
        except Exception as e:
            print(f"ERR: {e}")
            rows.append({"Ticker": tk.split('.', 1)[0], "ERROR": str(e)})
        _tm.sleep(PAUSE_SEC)
    return _pd.DataFrame(rows)


def save(df: _pd.DataFrame, fname="eps_trend.csv"):
    if fname.lower().endswith(".xlsx"):
        df.to_excel(fname, index=False, engine="openpyxl")
    else:
        df.to_csv(fname, index=False, encoding="utf-8")
    print(f"\n✅  Saved {len(df)} rows → {fname}")


if __name__ == "__main__":
    raw = [
        "NET", "CRS", "ORLY", "DOCN", "TEM", "VST", "CVS", "TER", "RIVN", "NTGR",
        "SPOT", "UBER", "SNOW", "ACHR", "ASAN", "RKLB", "DPZ", "LYFT", "NEGG", "MSFT",
        "AAPL", "ARM", "AMZN", "META", "NFLX", "NVDA", "GOOG", "RDFN", "MDB", "Z",
        "INTC", "AMD", "AFRM", "COIN", "TSLA", "HOOD", "CAVA", "F", "GM", "SMR", "U",
        "RDDT", "SHOP", "SOFI"
    ]
    tickers = [f"{t}.US" for t in raw]
    tbl = build_table(tickers)
    save(tbl, "eps_trend.xlsx")


TMDX.US ...ok
JAMF.US ...ok
INZY.US ...ok
VSEC.US ...ok
KRKR.US ...ok
KOPN.US ...ok
MIST.US ...ok
THRD.US ...ok
TIO.US ...ok
DNMR.US ...ok

✅  Saved 10 rows → eps_trend.xlsx
